The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:

00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010

Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.

The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.

The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.

So, the gamma rate is the binary number 10110, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [1]:
with open('./inputs/03_bits.txt','r') as infile:
    bits = []
    for l in infile.readlines():
        bits.append(l.strip())

In [3]:
len(bits)

1000

In [24]:
from bitarray import bitarray
from bitarray.util import ba2int

import numpy as np

In [66]:
def binlist_power(inlist):
    # NB: won't work well if each value is equally common in a given place!
    thresh = len(inlist)/2
    
    bit_arr = np.array([list(t) for t in inlist]).astype(int)
    bitsum = sum(bit_arr)
    
    gbit = bitarray([x > thresh for x in bitsum])
    ebit = ~gbit
    
    gamma = ba2int(gbit)
    epsilon = ba2int(ebit)
    
    return gamma, epsilon

In [68]:
g,e = binlist_power(bits)
print('Gamma:   ',g)
print('Epsilon: ',e)
print('Power:   ',g*e)

Gamma:    844
Epsilon:  3251
Power:    2743844


<ipython-input-66-6cf0610f2ff5>:8: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  gbit = bitarray([x > thresh for x in bitsum])


---

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:

    Keep only numbers selected by the bit criteria for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
    If you only have one number left, stop; this is the rating value for which you are searching.
    Otherwise, repeat the process, considering the next bit to the right.

The bit criteria depends on which type of rating value you want to find:

    To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
    To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.


Finally, to find the life support rating, multiply the oxygen generator rating by the CO2 scrubber rating.

In [69]:
toylist = [
'00100',
'11110',
'10110',
'10111',
'10101',
'01111',
'00111',
'11100',
'10000',
'11001',
'00010',
'01010'
]

In [88]:
def oxlist(inlist,posn=0):
    if len(inlist) <= 1:
        return inlist
    else:
        l1 = [ t for t in inlist if t[posn] == '1']
        l0 = [ t for t in inlist if t[posn] == '0']
        
        if len(l1) >= len(l0):
            return oxlist(l1,posn=posn+1)
        
        else:
            return oxlist(l0,posn=posn+1)

In [89]:
def colist(inlist,posn=0):
    if len(inlist) == 1:
        return inlist
    else:
        l1 = [ t for t in inlist if t[posn] == '1']
        l0 = [ t for t in inlist if t[posn] == '0']
        
        if len(l1) < len(l0):
            return colist(l1,posn=posn+1)
        
        else:
            return colist(l0,posn=posn+1)

In [90]:
def lifesupport(inlist):
    ox = ba2int(bitarray(oxlist(inlist)[0]))
    co = ba2int(bitarray(colist(inlist)[0]))
    return ox * co 

In [92]:
lifesupport(bits)

6677951